In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.preprocessing import OrdinalEncoder

In [2]:
covid_df = pd.read_csv("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv")


/Users/arnieacevedo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
covid_df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-05,KY,21.0,NELSON,21179.0,NaN,NaN,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Yes,Missing,NaN,NaN
1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,2020-12,MI,26.0,HURON,26063.0,18 to 49 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN,NaN
4,2020-05,WI,55.0,POLK,55095.0,18 to 49 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887798,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887799,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26887800,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887801,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [4]:
covid_df["res_state"].value_counts()

CA    3694093
NY    2174580
FL    1709355
IL    1389929
PA    1205350
OH    1106582
NJ    1057775
NC    1006752
GA     905901
AZ     884880
TN     814932
IN     748025
MA     720824
VA     672204
MI     615507
MN     603542
SC     601374
WI     575946
AL     531029
CO     486268
OK     446452
WA     442581
MD     417253
UT     410989
IA     377825
CT     324615
KS     317817
NV     305455
KY     274133
AR     263249
OR     199354
ID     193311
NE     192218
NM     176842
SD     113731
MT     111522
RI     107745
DE     105855
NH      81076
TX      80076
ME      68306
AK      68020
MO      60247
MS      53227
DC      48511
HI      28125
PR      27370
ND      24174
VT      22338
LA      19834
WV       8691
GU       7491
VI       2588
WY       1284
Name: res_state, dtype: int64

In [5]:
#Seeing what dropping NA variables will do Source:https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe

def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(covid_df)

Your selected dataframe has 19 columns.
There are 12 columns that have missing values.


,Missing Values,% of Total Values
underlying_conditions_yn,25104177,93.4
case_positive_specimen_interval,19567816,72.8
case_onset_interval,14584639,54.2
ethnicity,5483645,20.4
race,4573908,17.0
res_county,1832227,6.8
county_fips_code,1832227,6.8
death_yn,1314646,4.9
sex,801502,3.0
age_group,305828,1.1


In [6]:
#Tells the spread of data 

covid_df["age_group"].value_counts()

18 to 49 years    14272548
50 to 64 years     5334509
65+ years          3538828
0 - 17 years       3263868
Missing             172222
Name: age_group, dtype: int64

In [7]:
covid_df["death_yn"].value_counts()

No         12751685
Missing     9666074
Unknown     2876301
Yes          279097
Name: death_yn, dtype: int64

In [8]:
age_covid_df = covid_df.dropna(subset=["age_group", "death_yn"]).reset_index()

age_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
1,2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
2,5,2021-01,IL,17.0,RANDOLPH,17157.0,18 to 49 years,Missing,Missing,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
3,7,2020-11,MN,27.0,BECKER,27005.0,0 - 17 years,Female,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
4,8,2020-10,NY,36.0,CORTLAND,36023.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326329,26887798,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
25326330,26887799,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
25326331,26887800,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
25326332,26887801,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [9]:
age_covid_df.drop(age_covid_df.index[age_covid_df['death_yn'] == 'Missing'], inplace = True)

age_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
1,2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,7,2020-11,MN,27.0,BECKER,27005.0,0 - 17 years,Female,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
4,8,2020-10,NY,36.0,CORTLAND,36023.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
5,9,2021-01,ME,23.0,CUMBERLAND,23005.0,0 - 17 years,Female,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25325798,26887266,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326015,26887483,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326073,26887541,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326299,26887768,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN


In [10]:
age_covid_df.drop(age_covid_df.index[age_covid_df['death_yn'] == 'Unknown'], inplace = True)

age_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
1,2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,7,2020-11,MN,27.0,BECKER,27005.0,0 - 17 years,Female,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
4,8,2020-10,NY,36.0,CORTLAND,36023.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
5,9,2021-01,ME,23.0,CUMBERLAND,23005.0,0 - 17 years,Female,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25325798,26887266,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326015,26887483,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326073,26887541,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326299,26887768,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN


In [11]:
age_covid_df.drop(age_covid_df.index[age_covid_df['age_group'] == 'Missing'], inplace = True)

age_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
1,2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,7,2020-11,MN,27.0,BECKER,27005.0,0 - 17 years,Female,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
4,8,2020-10,NY,36.0,CORTLAND,36023.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
5,9,2021-01,ME,23.0,CUMBERLAND,23005.0,0 - 17 years,Female,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25325798,26887266,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326015,26887483,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326073,26887541,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
25326299,26887768,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN


In [12]:
age_covid_df.reset_index(drop=True, inplace=True)

age_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
1,2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
2,7,2020-11,MN,27.0,BECKER,27005.0,0 - 17 years,Female,Unknown,Missing,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
3,8,2020-10,NY,36.0,CORTLAND,36023.0,0 - 17 years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
4,9,2021-01,ME,23.0,CUMBERLAND,23005.0,0 - 17 years,Female,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12891577,26887266,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
12891578,26887483,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
12891579,26887541,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN
12891580,26887768,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,NaN


In [13]:
#age_covid_df.to_csv("/Users/arnieacevedo/Desktop/Class Project/age_covid_df.csv")
#After doing further scrubbing our team realized we needed to also drop the missing and unknown rows for more accurate data
#this also meant that this data frame no longer would be viable for our machine learning

In [13]:
age_covid_df.drop(['state_fips_code', 'res_county', 'county_fips_code', 'case_positive_specimen_interval', 'case_onset_interval', 'process', 'current_status', 'underlying_conditions_yn'], axis = 1, inplace = True)


In [14]:
age_covid_df

,index,case_month,res_state,age_group,sex,race,ethnicity,exposure_yn,symptom_status,hosp_yn,icu_yn,death_yn
0,1,2020-12,OH,0 - 17 years,NaN,NaN,NaN,Unknown,Symptomatic,No,Missing,No
1,2,2021-05,OH,0 - 17 years,NaN,NaN,NaN,Unknown,Symptomatic,No,No,No
2,7,2020-11,MN,0 - 17 years,Female,Unknown,Missing,Missing,Missing,Missing,Missing,No
3,8,2020-10,NY,0 - 17 years,Female,NaN,NaN,Missing,Missing,Missing,Missing,No
4,9,2021-01,ME,0 - 17 years,Female,NaN,NaN,Missing,Missing,No,Missing,No
...,...,...,...,...,...,...,...,...,...,...,...,...
12891577,26887266,2020-12,IN,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,No,Missing,No
12891578,26887483,2020-12,IN,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,No,Missing,No
12891579,26887541,2020-12,IN,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,No,Missing,No
12891580,26887768,2020-12,IN,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,No,Missing,No


In [16]:
#Commented out for time when rerunning the code
#age_covid_df.to_csv("/Users/arnieacevedo/Desktop/Class Project/small2_covid_df.csv")

In [15]:
age_covid_df["res_state"].value_counts()

CA    3662903
NY    1069046
NJ    1029244
NC     976579
FL     869400
OH     591535
MN     589275
AZ     325054
KS     308287
MI     293175
SC     285549
VA     276315
OK     255236
KY     238260
PA     230879
WI     199817
NV     195303
UT     192141
OR     156236
TN     124101
MA     119959
AR     117210
ID     115372
AL      91169
CO      80508
IA      76888
MT      70264
ME      64566
DC      46080
NH      39258
MS      31410
RI      26101
CT      25214
IN      24532
IL      22507
VT      18135
MD      13243
NM      12507
GA       6377
LA       5901
ND       4961
PR       2536
AK       2445
WA       2129
VI       2045
GU       1265
WY        494
MO         67
DE         49
WV         44
Name: res_state, dtype: int64

In [16]:
test_covid_df = age_covid_df[['age_group', 'death_yn']]

test_covid_df

,age_group,death_yn
0,0 - 17 years,No
1,0 - 17 years,No
2,0 - 17 years,No
3,0 - 17 years,No
4,0 - 17 years,No
...,...,...
12891577,18 to 49 years,No
12891578,18 to 49 years,No
12891579,18 to 49 years,No
12891580,18 to 49 years,No


In [19]:
#test_covid_df.to_csv("/Users/arnieacevedo/Desktop/Class Project/test_covid_df.csv")

In [35]:
fl_test_covid_df = age_covid_df.loc[age_covid_df['res_state'] == 'FL']

fl_test_covid_df

,index,case_month,res_state,age_group,sex,race,ethnicity,exposure_yn,symptom_status,hosp_yn,icu_yn,death_yn
53,151,2021-01,FL,0 - 17 years,Female,White,Hispanic/Latino,Missing,Symptomatic,No,Missing,No
67,188,2020-09,FL,18 to 49 years,Female,NaN,NaN,Missing,Missing,No,Missing,No
114,317,2021-01,FL,0 - 17 years,Female,White,Hispanic/Latino,Missing,Symptomatic,No,Missing,No
120,323,2020-11,FL,18 to 49 years,Female,White,Hispanic/Latino,Missing,Symptomatic,No,Missing,No
142,400,2020-07,FL,0 - 17 years,Female,White,Hispanic/Latino,Missing,Symptomatic,No,Missing,No
...,...,...,...,...,...,...,...,...,...,...,...,...
11906117,25114205,2020-07,FL,18 to 49 years,Male,White,Unknown,Missing,Missing,Missing,Missing,No
11906124,25114213,2020-07,FL,18 to 49 years,Male,White,Unknown,Missing,Symptomatic,No,Missing,No
11906128,25114217,2020-07,FL,18 to 49 years,Male,White,Unknown,Missing,Missing,No,Missing,No
11906132,25114221,2020-07,FL,18 to 49 years,Male,White,Unknown,Missing,Symptomatic,No,Missing,No


In [18]:
fl_test_covid_df = fl_test_covid_df[['age_group', 'death_yn']]

fl_test_covid_df.reset_index(drop=True, inplace=True)

fl_test_covid_df

,age_group,death_yn
0,0 - 17 years,No
1,18 to 49 years,No
2,0 - 17 years,No
3,18 to 49 years,No
4,0 - 17 years,No
...,...,...
869395,18 to 49 years,No
869396,18 to 49 years,No
869397,18 to 49 years,No
869398,18 to 49 years,No


In [22]:
#fl_test_covid_df.to_csv("/Users/arnieacevedo/Desktop/Class Project/fl_test_covid_df.csv")

In [19]:
# Source for random forest information https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

clean_covid_df = covid_df.dropna().reset_index()

clean_covid_df

,index,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,80,2021-04,NC,37.0,CLEVELAND,37045.0,18 to 49 years,Female,White,Hispanic/Latino,0.0,0.0,Missing,Unknown,Laboratory-confirmed case,Symptomatic,No,Unknown,No,Yes
1,157,2020-12,MN,27.0,WASHINGTON,27163.0,18 to 49 years,Female,White,Hispanic/Latino,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,Yes
2,220,2020-12,MN,27.0,FILLMORE,27045.0,50 to 64 years,Female,White,Missing,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,Yes
3,249,2020-12,MN,27.0,SCOTT,27139.0,18 to 49 years,Male,White,Hispanic/Latino,0.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,Yes
4,387,2020-12,NY,36.0,QUEENS,36081.0,50 to 64 years,Female,American Indian/Alaska Native,Hispanic/Latino,0.0,0.0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675368,23712925,2021-02,NY,36.0,KINGS,36047.0,65+ years,Male,White,Unknown,0.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,Yes
675369,23712933,2021-02,NY,36.0,KINGS,36047.0,65+ years,Male,White,Unknown,1.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,Yes
675370,23712941,2021-02,NY,36.0,KINGS,36047.0,65+ years,Male,White,Unknown,0.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,Yes
675371,23712965,2021-02,NY,36.0,KINGS,36047.0,65+ years,Male,White,Unknown,1.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,Yes


In [24]:
#Loading the dataset
#df = clean_covid_df
#X = df.drop("death_yn",1)   #Feature Matrix
#y = df["death_yn"]          #Target Variable


In [25]:
#Using Pearson Correlation
#plt.figure(figsize=(12,10))
#cor = df.corr()
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

In [26]:
#train

In [27]:
#target

In [28]:
#from sklearn.preprocessing import StandardScaler,OneHotEncoder
#import tensorflow as tf

In [29]:
# Generate our categorical variable list
#attrition_cat = df.dtypes[df.dtypes == "object"].index.tolist()
#attrition_cat

In [30]:
# Check the number of unique values in each column
#df[attrition_cat].nunique()

In [31]:
# Create a OneHotEncoder instance
#enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
#encode_df = pd.DataFrame(enc.fit_transform(df[attrition_cat]))

# Add the encoded variable names to the DataFrame
#encode_df.columns = enc.get_feature_names(attrition_cat)
#encode_df.head()

In [32]:
#Using Pearson Correlation
#plt.figure(figsize=(12,10))
#cor = encode_df.corr()
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

In [33]:
#Using Pearson Correlation
#plt.figure
#cor = encode_df.corr()
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#plt.show()

In [20]:
covid_df["age_group"].value_counts()

18 to 49 years    14272548
50 to 64 years     5334509
65+ years          3538828
0 - 17 years       3263868
Missing             172222
Name: age_group, dtype: int64

In [21]:
split_age_df = covid_df[(covid_df['age_group'] == '0 - 17 years') | (covid_df['age_group'] == '18 to 49 years') | (covid_df['age_group'] == '50 to 64 years')]

split_age_df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
1,2020-12,OH,39.0,VAN WERT,39161.0,0 - 17 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
2,2021-05,OH,39.0,WYANDOT,39175.0,0 - 17 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,2020-12,MI,26.0,HURON,26063.0,18 to 49 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN,NaN
4,2020-05,WI,55.0,POLK,55095.0,18 to 49 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,NaN,NaN
5,2021-01,IL,17.0,RANDOLPH,17157.0,18 to 49 years,Missing,Missing,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887798,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887799,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26887800,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887801,2020-12,IN,18.0,NaN,NaN,18 to 49 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [22]:
split_age_df2 = covid_df[(covid_df['age_group'] == '65+ years')]

split_age_df2

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
30,2020-12,OH,39.0,CRAWFORD,39033.0,65+ years,Female,NaN,NaN,NaN,0.0,Unknown,Unknown,Probable Case,Symptomatic,No,No,NaN,NaN
31,2021-01,NY,36.0,ESSEX,36031.0,65+ years,Female,NaN,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
32,2020-11,OH,39.0,JEFFERSON,39081.0,65+ years,Female,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN
33,2021-03,PA,42.0,MCKEAN,42083.0,65+ years,Female,NaN,NaN,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,No,Unknown,NaN,Yes
52,2021-01,SC,45.0,FLORENCE,45041.0,65+ years,Female,Black,Missing,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26884256,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26884257,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26884258,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26884259,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [23]:
split_age_df['age_group'].value_counts()

18 to 49 years    14272548
50 to 64 years     5334509
0 - 17 years       3263868
Name: age_group, dtype: int64

In [24]:
split_age_df2['age_group'].value_counts()

65+ years    3538828
Name: age_group, dtype: int64

In [25]:
split_age_df['age_group'] = "0 to 64 years"

<ipython-input-25-7677b8026e8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_age_df['age_group'] = "0 to 64 years"


In [26]:
split_age_df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
1,2020-12,OH,39.0,VAN WERT,39161.0,0 to 64 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
2,2021-05,OH,39.0,WYANDOT,39175.0,0 to 64 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,2020-12,MI,26.0,HURON,26063.0,0 to 64 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN,NaN
4,2020-05,WI,55.0,POLK,55095.0,0 to 64 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,NaN,NaN
5,2021-01,IL,17.0,RANDOLPH,17157.0,0 to 64 years,Missing,Missing,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26887798,2020-12,IN,18.0,NaN,NaN,0 to 64 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887799,2020-12,IN,18.0,NaN,NaN,0 to 64 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26887800,2020-12,IN,18.0,NaN,NaN,0 to 64 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26887801,2020-12,IN,18.0,NaN,NaN,0 to 64 years,Male,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [51]:
final_split_age_df = pd.concat([split_age_df, split_age_df2])

final_split_age_df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
1,2020-12,OH,39.0,VAN WERT,39161.0,0 to 64 years,NaN,NaN,NaN,NaN,0.0,Clinical evaluation,Unknown,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
2,2021-05,OH,39.0,WYANDOT,39175.0,0 to 64 years,NaN,NaN,NaN,0.0,0.0,Clinical evaluation,Unknown,Probable Case,Symptomatic,No,No,No,NaN
3,2020-12,MI,26.0,HURON,26063.0,0 to 64 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN,NaN
4,2020-05,WI,55.0,POLK,55095.0,0 to 64 years,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Unknown,Missing,NaN,NaN
5,2021-01,IL,17.0,RANDOLPH,17157.0,0 to 64 years,Missing,Missing,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26884256,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
26884257,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26884258,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
26884259,2021-05,IN,18.0,NaN,NaN,65+ years,Female,Unknown,Unknown,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [52]:
final_split_age_df.drop(['case_month', 'state_fips_code', 'county_fips_code', 'race', 'case_positive_specimen_interval', 'case_onset_interval', 'process', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn', 'underlying_conditions_yn'], axis = 1, inplace = True)

In [53]:
final_split_age_df = final_split_age_df.loc[final_split_age_df['res_state'] == 'FL'].reset_index()

final_split_age_df

,index,res_state,res_county,age_group,sex,ethnicity,exposure_yn,death_yn
0,62,FL,HILLSBOROUGH,0 to 64 years,Female,Hispanic/Latino,Missing,Missing
1,72,FL,SARASOTA,0 to 64 years,Female,Hispanic/Latino,Missing,Missing
2,99,FL,COLUMBIA,0 to 64 years,Female,NaN,Missing,NaN
3,151,FL,INDIAN RIVER,0 to 64 years,Female,Hispanic/Latino,Missing,No
4,188,FL,GADSDEN,0 to 64 years,Female,NaN,Missing,No
...,...,...,...,...,...,...,...,...
1697477,25114080,FL,NaN,65+ years,Male,Non-Hispanic/Latino,Missing,No
1697478,25114085,FL,NaN,65+ years,Male,Non-Hispanic/Latino,Missing,NaN
1697479,25114088,FL,NaN,65+ years,Male,Non-Hispanic/Latino,Missing,No
1697480,25114092,FL,NaN,65+ years,Male,Non-Hispanic/Latino,Missing,No


In [54]:
final_split_age_df.drop(['index', 'res_state', 'exposure_yn'], axis = 1, inplace = True)

final_split_age_df

,res_county,age_group,sex,ethnicity,death_yn
0,HILLSBOROUGH,0 to 64 years,Female,Hispanic/Latino,Missing
1,SARASOTA,0 to 64 years,Female,Hispanic/Latino,Missing
2,COLUMBIA,0 to 64 years,Female,NaN,NaN
3,INDIAN RIVER,0 to 64 years,Female,Hispanic/Latino,No
4,GADSDEN,0 to 64 years,Female,NaN,No
...,...,...,...,...,...
1697477,NaN,65+ years,Male,Non-Hispanic/Latino,No
1697478,NaN,65+ years,Male,Non-Hispanic/Latino,NaN
1697479,NaN,65+ years,Male,Non-Hispanic/Latino,No
1697480,NaN,65+ years,Male,Non-Hispanic/Latino,No


In [40]:
final_split_age_df['age_group'].value_counts()

0 to 64 years    22870925
65+ years         3538828
Name: age_group, dtype: int64

In [47]:
final_split_age_df['ethnicity'].value_counts()

Non-Hispanic/Latino    773591
Hispanic/Latino        432833
Unknown                220207
Missing                  1803
Name: ethnicity, dtype: int64

In [55]:
final_split_age_df['sex'].value_counts()

Female     883741
Male       793815
Unknown      2824
Missing      1487
Name: sex, dtype: int64

In [56]:
final_split_age_df['death_yn'].value_counts()

No         843590
Missing    723969
Yes         25810
Unknown       106
Name: death_yn, dtype: int64

In [59]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['ethnicity'] == 'Missing'], inplace = True)



In [62]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['ethnicity'] == 'Unknown'], inplace = True)


In [63]:
final_split_age_df['ethnicity'].value_counts()

Non-Hispanic/Latino    773591
Hispanic/Latino        432833
Name: ethnicity, dtype: int64

In [64]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['sex'] == 'Unknown'], inplace = True)

In [66]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['sex'] == 'Missing'], inplace = True)

In [67]:
final_split_age_df['sex'].value_counts()

Female    779626
Male      679125
Name: sex, dtype: int64

In [68]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['death_yn'] == 'Missing'], inplace = True)

In [69]:
final_split_age_df.drop(final_split_age_df.index[final_split_age_df['death_yn'] == 'Unknown'], inplace = True)

In [70]:
final_split_age_df['death_yn'].value_counts()

No     796004
Yes     25429
Name: death_yn, dtype: int64

In [71]:
final_split_age_df.dropna()

,res_county,age_group,sex,ethnicity,death_yn
3,INDIAN RIVER,0 to 64 years,Female,Hispanic/Latino,No
7,HERNANDO,0 to 64 years,Female,Hispanic/Latino,No
8,HENDRY,0 to 64 years,Female,Hispanic/Latino,No
9,PUTNAM,0 to 64 years,Female,Hispanic/Latino,No
10,CHARLOTTE,0 to 64 years,Female,Hispanic/Latino,No
...,...,...,...,...,...
1693902,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,Yes
1693905,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,No
1693911,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,No
1693913,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,Yes


In [72]:
final_split_age_df.to_csv("/Users/arnieacevedo/Desktop/Class Project/final_split_age_df.csv")